In [ ]:
#!pip install intuit-oauth
#!pip install selenium

# Selenium requires a driver to interface with the chosen browser. 
# Chrome driver https://sites.google.com/a/chromium.org/chromedriver/downloads

## Setup
https://developer.intuit.com/app/developer/qbpayments/docs/develop/authentication-and-authorization


In [12]:
#setup client_id (application id)
client_id = ''

#setup client_secret (application secret)
client_secret = '' 

#setup redirect_url (application redirect url)
redirect_uri = 'https://appcenter.intuit.com/'

#setup environment sandbox/production
environment = 'sandbox' #'production'

In [13]:
from intuitlib.client import AuthClient
from intuitlib.enums import Scopes
from selenium import webdriver
import time
import datetime
import keyring
import getpass
import pandas as pd
import requests


assert client_id, 'client_id is empty'
assert client_secret, 'client_secret is empty'
assert redirect_uri, 'redirect_url is empty'

if environment == 'sandbox':
    baseURL = 'https://sandbox-quickbooks.api.intuit.com'
else:
    baseURL = 'https://quickbooks.api.intuit.com'

# Prepare scopes
scopes = [Scopes.ACCOUNTING]

auth_client = AuthClient(
        client_id,
        client_secret,
        redirect_uri,
        environment,
    )

#refresh token max age (days)
token_max_age = 100

In [14]:
def FirstAuth():
    # Get authorization URL
    auth_url = auth_client.get_authorization_url(scopes)

    # Chrome webdriver      
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(auth_url)
    auth_res_url = auth_url
    #waiting while there will be "code" in the browser's URL
    while auth_res_url.find('code=') == -1:
        time.sleep(1) # 1 second sleep
        auth_res_url = driver.current_url        

    start_number = auth_res_url.find('code=') + len('code=')
    end_number = auth_res_url.find('&state=')
    auth_code = auth_res_url[start_number:end_number]

    start_number = auth_res_url.find('realmId=') + len('realmId=')
    realm_id = auth_res_url[start_number:]
    
    #print(f'auth_code = {auth_code}')
    #print(f'realm_id = {realm_id}')
    
    # Get OAuth2 Bearer token
    auth_client.get_bearer_token(auth_code, realm_id=realm_id)
    
    #print(f'access_token = {auth_client.access_token}')
    #print(f'refresh_token = {auth_client.refresh_token}')
        
    # Return tokens
    return realm_id

# Data API
def GetData(api_url, data_type):  
      
    last_page = False
    ret = dict()
    while not last_page: 
        response = requests.get(api_url, 
                               headers = {
                                   'Authorization': 'Bearer ' + auth_client.access_token,
                                   'Accept': 'application/json'
                               })
       
        #print(response)
        
        if response.status_code!=200:
            print(f'Bad server response for /{data_type}/ : {response}')
            if response.status_code not in (401, 403, 404, 502):
                print(f'Retrieved JSON:\n{response.json()}')
            return ret;    
        else:
            json_response = response.json()
           
            data_response = json_response[data_type]
            
            ret = data_response 
            
            last_page=True
        
    return ret

In [15]:
refresh_token_age = -1
token_date = keyring.get_password('qb_refresh_token_date', getpass.getuser())

if token_date is not None:
    token_date = datetime.datetime.fromtimestamp(int(float(token_date)))
    #print(f'refresh_token_date = {token_date}')
    current_date = datetime.datetime.now() 
    refresh_token_age = (current_date - token_date).days
    #print(f'refresh_token_age = {refresh_token_age}')
    
if (refresh_token_age < 0) | (refresh_token_age > token_max_age):    
    realm_id = FirstAuth()
    keyring.set_password('qb_realm_id', getpass.getuser(), realm_id)    
else:
    auth_client.refresh_token = keyring.get_password("qb_refresh_token", getpass.getuser())
    realm_id = keyring.get_password("qb_realm_id", getpass.getuser())

#refresh tokens
auth_client.refresh()
#save tokens
keyring.set_password('qb_refresh_token', getpass.getuser(), auth_client.refresh_token)
keyring.set_password('qb_refresh_token_date', getpass.getuser(), str(datetime.datetime.now().timestamp()))

#print(f'access_token = {auth_client.access_token}')
#print(f'refresh_token = {auth_client.refresh_token}')


api_uri = baseURL+'/v3/company/'+realm_id;
accounts_select = 'select * from Account where Metadata.CreateTime > \'2014-12-31\''
accounts_URL = api_uri+'/query?query='+accounts_select 
client = '1';
ProfitAndLoss_URL = api_uri+'/reports/ProfitAndLoss?name='+client
JournalReport_URL = api_uri+'/reports/JournalReport?name='+client


df_accounts = pd.DataFrame(GetData(accounts_URL, 'QueryResponse'))
df_profit_loss = pd.DataFrame(GetData(ProfitAndLoss_URL, 'Header'))
df_journal_report = pd.DataFrame(GetData(JournalReport_URL, 'Header'))

In [16]:
df_accounts

,Account,startPosition,maxResults
0,"{'Name': 'Accounts Payable (A/P)', 'SubAccount...",1,89
1,"{'Name': 'Accounts Receivable (A/R)', 'SubAcco...",1,89
2,"{'Name': 'Advertising', 'SubAccount': False, '...",1,89
3,"{'Name': 'Arizona Dept. of Revenue Payable', '...",1,89
4,"{'Name': 'Automobile', 'SubAccount': False, 'F...",1,89
...,...,...,...
84,"{'Name': 'Undeposited Funds', 'SubAccount': Fa...",1,89
85,"{'Name': 'Utilities', 'SubAccount': False, 'Fu...",1,89
86,"{'Name': 'Gas and Electric', 'SubAccount': Tru...",1,89
87,"{'Name': 'Telephone', 'SubAccount': True, 'Par...",1,89


In [17]:
df_profit_loss

,Time,ReportName,DateMacro,ReportBasis,StartPeriod,EndPeriod,SummarizeColumnsBy,Currency,Option
0,2020-10-17T20:40:53-07:00,ProfitAndLoss,this calendar year-to-date,Accrual,2020-01-01,2020-10-17,Total,USD,"{'Name': 'AccountingStandard', 'Value': 'GAAP'}"
1,2020-10-17T20:40:53-07:00,ProfitAndLoss,this calendar year-to-date,Accrual,2020-01-01,2020-10-17,Total,USD,"{'Name': 'NoReportData', 'Value': 'false'}"


In [18]:
df_journal_report

,Time,ReportName,DateMacro,StartPeriod,EndPeriod,Currency,Option
0,2020-10-17T20:40:53-07:00,JournalReport,this month-to-date,2020-10-01,2020-10-17,USD,"{'Name': 'NoReportData', 'Value': 'true'}"


### In case if needed to clear saved passwords

In [6]:
#keyring.delete_password("qb_refresh_token", getpass.getuser())
#keyring.delete_password("qb_refresh_token_date", getpass.getuser())
#keyring.delete_password("qb_realm_id", getpass.getuser())